In [ ]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [61]:
from transformers import T5ForConditionalGeneration, AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
from chat_template_utils import get_json_schema
from llama_cpp import Llama
from utils import *
from peft import *
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)


0it [00:00, ?it/s]

In [10]:
data = pd.read_excel('forms.xlsx',index_col=0)
data

,form
0,"<form class=""flex w-full max-w-xl flex-col gap..."
1,"<form accept-charset=""UTF-8"" action=""https://c..."
2,"<form data-testid=""ApplyStep1Form"" method=""POS..."
3,"<form novalidate="""">\n <div class=""sc-uJiQo eq..."
4,"<form class=""elementor-form"" method=""post"" nam..."
...,...
169,"<form class="""" method=""post"" action=""https://c..."
170,"<form id=""careerform"" name=""careerform"" action..."
171,"<form class=""mod-footer__newsletter-form hidde..."
172,"<form class=""form form--methods clearfix"" id=""..."


In [167]:
def preprocess_form(form):
    if isinstance(form,str):
        form = BeautifulSoup(form,'html.parser')
        
    form:PageElement
    comments=form.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment.extract()
    all_tags = form.find_all()

    # Remove all tags except for 'link', 'input', and 'select' tags
    i=0
    for element in all_tags:
        element:PageElement
        for k in ['style','class','bounding_box_rect','is_clickable']:
            if k in element.attrs:
                    del element[k]
                    pass
           
        if 'type' in element.attrs and element.attrs['type']=='hidden':
            element.unwrap()
            pass
            
        
        #if element.name  in ['div','text']: 
        if element.name not in ['a', 'input', 'select','radio','button','textarea','checkbox','option']:
            
            element.unwrap()
            #print(element)
            pass
        else:
            #element['backend_node_id']=i
            pass

            
                
            i+=1
            
    #print(form.prettify() )
    return form
    
import re

def get_query_element(f):
    query_element=[]
    mytext  = ''
    for element in f:
        # Get the text before the element
        #print('-----------------------------')
        if element.name == None:
            t = element.text.replace('\n','').replace('\t','').strip()
            if t!='':
                mytext+=t+'\n '
        else:
            query_element.append({'query':mytext,'element': element})
            mytext =''
        
    return query_element


def get_query_html(f):
    f:PageElement
    childs = f.find_all(string=True,recursive=False)
    for c in childs:
        print('---------------------')
        print(c)
    pass
    
    
    
f = preprocess_form(data['form'][141])

d = get_query_element(f)
print(d)

[{'query': 'nach stichwort suchen\n ', 'element': <input aria-label="nach stichwort suchen" maxlength="50" name="q" type="text"/>}, {'query': '', 'element': <a aria-controls="moreoptionsdiv" aria-expanded="false" data-lessoptions="weniger optionen anzeigen" data-moreoptions="mehr optionen anzeigen" data-target=".optionsdiv" data-toggle="collapse" href="javascript:void(0)" id="options-search" rel="advance" role="button">mehr optionen anzeigen</a>}, {'query': 'laden...\n job function\n ', 'element': <select id="optionsfacetsdd_customfield1" name="optionsfacetsdd_customfield1">
<option value="">alle</option>
</select>}, {'query': 'land/region\n ', 'element': <select id="optionsfacetsdd_country" name="optionsfacetsdd_country">
<option value="">alle</option>
</select>}, {'query': '', 'element': <input type="submit" value="stellen suchen"/>}, {'query': '', 'element': <a href="" role="button">löschen</a>}]


In [95]:
# Iterate over all elements




[{'query': '',
  'element': <button ng-class='candidate &amp;&amp; candidate.resume.file_name ? "gray" : "blue"' ng-click="$event.preventDefault(); showFileSelector();" ng-disabled="uploadingResume"> Upload Resume*</button>},
 {'query': '',
  'element': <input accept=".pdf, .doc, .docx, .txt, .rtd, .pages" id="main-attachment" name="cResume" ng-file-select="onFileSelect($files)" ng-model="candidate.resume" required="" type="file"/>},
 {'query': 'A resume is required The file attachment is limited to 50MB Personal Details Full Name * ',
  'element': <input name="cName" ng-model="candidate.name" placeholder="Full Name" required="" type="text"/>},
 {'query': 'A full name is required Email Address * ',
  'element': <input name="cEmail" ng-model="candidate.email_address" placeholder="Email Address" required="" type="email"/>},
 {'query': 'An email is required An email is required Phone Number ',
  'element': <input name="cPhoneNumber" ng-model="candidate.phone_number" placeholder="Phone Num

In [57]:
with open('./html/1.html','r',encoding='utf-8') as f:
    s= f.read()
    
preprocess_form(s)

<form name="form" ng-class='formSubmitted ? "formSubmitted" : ""' ng-submit="apply()" novalidate="">
 <div>
  <ul>
   <li>
    <div>
     <button ng-class='candidate &amp;&amp; candidate.resume.file_name ? "gray" : "blue"' ng-click="$event.preventDefault(); showFileSelector();" ng-disabled="uploadingResume">
      <!-- ngIf: !uploadingResume && !candidate.resume.file_name -->
      <div ng-if="!uploadingResume &amp;&amp; !candidate.resume.file_name">
       <i>
       </i>
       <span>
       </span>
       <span>
        Upload Resume
       </span>
       <span ng-show="resumeRequired" title="Required">
        *
       </span>
      </div>
      <!-- end ngIf: !uploadingResume && !candidate.resume.file_name -->
      <!-- ngIf: uploadingResume -->
      <!-- ngIf: !uploadingResume && candidate.resume.file_name -->
     </button>
    </div>
   </li>
  </ul>
 </div>
 <div>
  <input id="resume_required" type="hidden" value="required"/>
  <div>
   <!-- ngIf: candidate.resume.file_name 

In [69]:
comments = f.find_all(string=lambda text: isinstance(text, Comment))
for comment in comments:
    comment.extract()
    
print(f.prettify())

<button ng-class='candidate &amp;&amp; candidate.resume.file_name ? "gray" : "blue"' ng-click="$event.preventDefault(); showFileSelector();" ng-disabled="uploadingResume">
 <span>
 </span>
 <span>
  Upload Resume
 </span>
 <span ng-show="resumeRequired" title="Required">
  *
 </span>
</button>
<input id="resume_required" type="hidden" value="required"/>
<input accept=".pdf, .doc, .docx, .txt, .rtd, .pages" id="main-attachment" name="cResume" ng-file-select="onFileSelect($files)" ng-model="candidate.resume" required="" type="file"/>
<span ng-show="resumeRequired &amp;&amp; !candidate.resume.file_name">
 A resume is required
</span>
<span>
 The file attachment is limited to 50MB
</span>
Personal Details
<span>
 Full Name
</span>
<span title="Required">
 *
</span>
<input name="cName" ng-model="candidate.name" placeholder="Full Name" required="" type="text"/>
<span ng-show="form.cName.$error.required">
 A full name is required
</span>
<span>
 Email Address
</span>
<span title="Required">
 